**Created on 13/01/2023**

**Authors: Thomas ROSTAINGT**

**License: this code is released under the CeCILL 2.1 license. See https://www.cecill.info/licenses/Licence_CeCILL_V2.1-en.txt**

In [1]:
# libraries pour créer des exemples
import numpy as np
import pandas as pd

## Importation des données

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
df = pd.read_csv('/content/drive/MyDrive/M2 SID/PIP2023/data_v2.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [32]:
# données fournies par le groupe 1
#--------------------------------#
df_train = pd.read_csv('/content/drive/MyDrive/M2 SID/PIP2023/Méthode Analytique/train_new_range.csv')
df_test = pd.read_csv('/content/drive/MyDrive/M2 SID/PIP2023/Méthode Analytique/test_new_range.csv')
df_val = pd.read_csv('/content/drive/MyDrive/M2 SID/PIP2023/Méthode Analytique/validation_new_range.csv')

df_new_range_skew = pd.concat([df_train,df_test,df_val])
df_new_range_skew = df_new_range_skew.drop(['Unnamed: 0'], axis=1)
df_new_range_skew.drop_duplicates(keep = 'first', inplace=True)

In [33]:
(abs(df['distance']-df['range_skew'])).mean() 
# erreur moyenne entre range_skew et distance réelle de 17,5 cm 

0.17512418449062345

In [34]:
# On remplace les valeurs de range_skew par les distances prédites par le groupe 1
liste_seq = sorted(df.seqnum.tolist())
l = []
for i in liste_seq:
  if i in df_new_range_skew.sort_values(by = 'seqnum').seqnum.tolist():
    a = float(df_new_range_skew[df_new_range_skew['seqnum'] == i].nouveau_range_skew)
    l.append(a)
  else :
    a = float(df[df['seqnum'] == i].range_skew)
    l.append(a)

df4 = pd.DataFrame({'seqnum' : liste_seq, 'nouveau_range_skew' : l})
df = pd.merge(df, df4, on = 'seqnum' , how = 'left')

In [35]:
(abs(df['distance']-df['nouveau_range_skew'])).mean()
# erreur moyenne entre nouveau_range_skew et distance réelle de 9,5 cm 

0.09593310015476841

In [36]:
capteurs = np.unique(df['target'])
for capt in capteurs:
  print('nombre de signaux reçus par le capeur',capt,':',len(df[df.target==capt]['seqnum']))
# le nombre de signaux reçus par les capteurs n'est pas le même
# certains capteurs n'ont pas reçu le signal émis par le client 

nombre de signaux reçus par le capeur 100 : 643
nombre de signaux reçus par le capeur 101 : 706
nombre de signaux reçus par le capeur 150 : 477
nombre de signaux reçus par le capeur 171 : 704
nombre de signaux reçus par le capeur 180 : 708
nombre de signaux reçus par le capeur 184 : 708


### Calcul de position 

In [37]:
def trackObject(x1,y1,r1,x2,y2,r2,x3,y3,r3):
  A = 2*x2 - 2*x1
  B = 2*y2 - 2*y1
  C = r1**2 - r2**2 - x1**2 + x2**2 - y1**2 + y2**2
  D = 2*x3 - 2*x2
  E = 2*y3 - 2*y2
  F = r2**2 - r3**2 - x2**2 + x3**2 - y2**2 + y3**2

  x = (C*E - F*B) / (E*A - B*D)
  y = (C*D - A*F) / (B*D - A*E)
  return x,y

def score_distanceM(Xobserve:float, Yobserve:float, Xprevu:float, Yprevu:float) -> float:
    X=(Xobserve-Xprevu)**2
    Y=(Yobserve-Yprevu)**2
    return np.mean(np.sqrt(X+Y))

In [38]:
df = df[df.target!=150] 
# on retire le capteur 150 car ses distances estimées (range_skew) sont trop éloignées de la distance réelle

In [42]:
from pandas._config import options
from itertools import combinations
import random
df2 = pd.DataFrame(columns=['seqnum1', 'x1', 'y1', 'r1', 'seqnum2', 'x2', 'y2', 'r2', 'seqnum3', 'x3', 'y3', 'r3', 'xTrue', 'yTrue', 'xCalc', 'yCalc'])

for i in range(len(df)):
    
    if (i%6 == 0):
        option1, option2, option3 = random.sample(range(i+1, i+7), 3)

        OK = False
        while OK == False:
            try : 
                r1 = abs(float(df[df.seqnum == option1].nouveau_range_skew))
                r2 = abs(float(df[df.seqnum == option2].nouveau_range_skew))
                r3 = abs(float(df[df.seqnum == option3].nouveau_range_skew))
                OK = True
            except:
                option1, option2, option3 = random.sample(range(i+1, i+7), 3)

        x1 = float(df[df.seqnum == option1].loc_target_x)
        y1 = float(df[df.seqnum == option1].loc_target_y)  
        x2 = float(df[df.seqnum == option2].loc_target_x)
        y2 = float(df[df.seqnum == option2].loc_target_y)  
        x3 = float(df[df.seqnum == option3].loc_target_x)
        y3 = float(df[df.seqnum == option3].loc_target_y) 
        xTrue = float(df[df.seqnum == option1].loc_initiator_x)
        yTrue = float(df[df.seqnum == option1].loc_initiator_y)
        
        xCalc, yCalc = trackObject(x1,y1,r1,x2,y2,r2,x3,y3,r3)
        
    df2 = df2.append({'x1':x1, 'y1':y1, 'r1':r1, 
                      'x2':x2, 'y2':y2, 'r2':r2, 
                      'x3':x3, 'y3':y3, 'r3':r3, 
                      'seqnum1': option1, 'seqnum2': option2, 'seqnum3': option3,
                      'xTrue':xTrue, 'yTrue':yTrue, 
                      'xCalc':xCalc, 'yCalc':yCalc} , ignore_index=True)

df2.drop_duplicates(keep = 'first', inplace=True)

In [43]:
df2

,seqnum1,x1,y1,r1,seqnum2,x2,y2,r2,seqnum3,x3,y3,r3,xTrue,yTrue,xCalc,yCalc
0,4.0,-3.962,7.931,8.000550,1.0,2.120,2.793,0.201406,6.0,-1.190,4.578,3.769005,2.120,2.793,2.045459,2.648471
6,10.0,-3.962,7.931,7.958623,11.0,-3.132,4.339,5.855792,8.0,6.169,7.657,6.204717,2.120,2.793,2.211252,3.421932
12,18.0,-1.190,4.578,3.644861,16.0,-3.962,7.931,7.909172,13.0,2.120,2.793,0.308668,2.120,2.793,1.857069,2.572267
18,22.0,-3.962,7.931,7.952794,23.0,-3.132,4.339,5.765985,20.0,6.169,7.657,6.225616,2.120,2.793,2.190141,3.284682
24,26.0,6.169,7.657,6.053102,25.0,2.120,2.793,0.113783,29.0,-3.132,4.339,5.946552,2.120,2.793,2.614252,2.733722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,3446.0,6.169,7.657,6.952573,3450.0,-1.190,4.578,3.302396,3449.0,-3.132,4.339,4.358619,-0.645,7.777,-0.376833,6.889542
3450,3452.0,6.169,7.657,6.882621,3456.0,-1.190,4.578,3.349691,3455.0,-3.132,4.339,4.393341,-0.645,7.777,-0.417056,7.193927
3456,3457.0,2.120,2.793,5.666413,3458.0,6.169,7.657,6.924110,3460.0,-3.962,7.931,3.240001,-0.645,7.777,-0.747932,7.669886
3462,3463.0,2.120,2.793,5.720634,3467.0,-3.132,4.339,4.345756,3468.0,-1.190,4.578,3.187374,-0.645,7.777,-0.399739,8.403073


In [45]:
print('erreur de distance moyenne :',score_distanceM(Xobserve = df2.xTrue,
                Yobserve = df2.yTrue,
                Xprevu = df2.xCalc,
                Yprevu = df2.yCalc)*100,'cm')

erreur de distance moyenne : 38.35828919668542 cm
